In [126]:
import os
import hardposit as hp
import softposit as sp
import fixed2float as fi
from pathlib import Path
import re
import ast

In [127]:
import fixed2float
fixed2float.__version__

'4.0.1'

In [128]:
N, ES = 16, 1

In [172]:
with open(Path("../tb_ppu.log"), 'r') as f:
  log = f.read()
print(log)

(0x00003524, 0x00005e81, 0x0000000284df2404, 0x00005427)
(0x0000d609, 0x00005663, 0x00000001623ce184, 0x00004624)
(0x00007b0d, 0x0000998d, 0xfffffa7ec75ce184, 0x000081d0)
(0x00008465, 0x00005212, 0xfffff875902ce184, 0x0000818f)
(0x0000e301, 0x0000cd0d, 0xfffff875aef56324, 0x0000818f)
(0x0000f176, 0x0000cd3d, 0xfffff875b6a05504, 0x0000818f)
(0x000057ed, 0x0000f78c, 0xfffff875a94ee884, 0x0000818f)
(0x0000e9f9, 0x000024c6, 0xfffff875a033bd34, 0x0000818f)
(0x000084c5, 0x0000d2aa, 0xfffff8d46505bd34, 0x0000819b)
(0x0000f7e5, 0x00007277, 0xfffff8d3fadb8934, 0x0000819a)



In [173]:
log.splitlines()

['(0x00003524, 0x00005e81, 0x0000000284df2404, 0x00005427)',
 '(0x0000d609, 0x00005663, 0x00000001623ce184, 0x00004624)',
 '(0x00007b0d, 0x0000998d, 0xfffffa7ec75ce184, 0x000081d0)',
 '(0x00008465, 0x00005212, 0xfffff875902ce184, 0x0000818f)',
 '(0x0000e301, 0x0000cd0d, 0xfffff875aef56324, 0x0000818f)',
 '(0x0000f176, 0x0000cd3d, 0xfffff875b6a05504, 0x0000818f)',
 '(0x000057ed, 0x0000f78c, 0xfffff875a94ee884, 0x0000818f)',
 '(0x0000e9f9, 0x000024c6, 0xfffff875a033bd34, 0x0000818f)',
 '(0x000084c5, 0x0000d2aa, 0xfffff8d46505bd34, 0x0000819b)',
 '(0x0000f7e5, 0x00007277, 0xfffff8d3fadb8934, 0x0000819a)']

In [174]:
log.splitlines()[0]

'(0x00003524, 0x00005e81, 0x0000000284df2404, 0x00005427)'

In [175]:
lst = list(map(lambda l: ast.literal_eval(l), log.splitlines()))

In [176]:
operand1_i, operand2_i, fixed_o, result_o = list(zip(*lst)) 

In [177]:
operand1_i, operand2_i, fixed_o, result_o

((13604, 54793, 31501, 33893, 58113, 61814, 22509, 59897, 33989, 63461),
 (24193, 22115, 39309, 21010, 52493, 52541, 63372, 9414, 53930, 29303),
 (10819150852,
  5943124356,
  18446738021150417284,
  18446735782546563460,
  18446735783063020324,
  18446735783191663876,
  18446735782968223876,
  18446735782815448372,
  18446736189844471092,
  18446736188063320372),
 (21543, 17956, 33232, 33167, 33167, 33167, 33167, 33167, 33179, 33178))

In [178]:
acc = hp.from_bits(0, N, ES)
for (operand1, operand2) in zip(operand1_i, operand2_i):
  acc += (hp.from_bits(operand1, N, ES) * hp.from_bits(operand2, N, ES))

In [179]:
acc

1111111001100101 P<16,1>(33179) -1832.0

In [180]:
M, B = 31, 64
fi.from_bits(fixed_o[-1], M, B)

1111111111111111111110001101001111111010110110111000100100110100 Fx<31,64>(18446736188063320372) -1836.0200876472518

In [181]:
hp.from_bits(result_o[-1], N, ES).eval()

-1840.0

### test result using softposit

In [182]:
softposit_acc = [0] * len(operand1_i)

q = sp.quire16()

for i, (operand1, operand2) in enumerate(zip(operand1_i, operand2_i)):
  q.qma(
    sp.posit16(bits=operand1), sp.posit16(bits=operand2)
  )
  softposit_acc[i] = q.v

In [183]:
softposit_acc

[2.51904296875,
 1.3837890625,
 -1408.0,
 -1928.0,
 -1928.0,
 -1928.0,
 -1928.0,
 -1928.0,
 -1832.0,
 -1840.0]

### FPPU hardware outputs

In [184]:
results_o = list(map(lambda bits: hp.from_bits(bits, N, ES).eval(), result_o))
results_o

[2.51904296875,
 1.3837890625,
 -1408.0,
 -1928.0,
 -1928.0,
 -1928.0,
 -1928.0,
 -1928.0,
 -1832.0,
 -1840.0]

In [185]:
list(map(lambda val1, val2: val1 == val2, map(str, results_o), map(str, softposit_acc)))

[True, True, True, True, True, True, True, True, True, True]

In [186]:
assert all(_) == True
print("Congratulation, it works.")

Congratulation, it works.
